In [1]:
import re
import os
import requests
import pandas as pd
import multiprocessing
import time
from time import time as timer
from tqdm import tqdm
import numpy as np
from pathlib import Path
from functools import partial
import requests
import urllib
from PIL import Image
import glob #Used to selecting images from a folder
import matplotlib.pyplot as plt

### *To Downolad the images from the data set we use this predefined code*

In [2]:
df=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/git/dataset.csv")

In [3]:
df.head()

,index,image_link,group_id,entity_name
0,1,https://m.media-amazon.com/images/I/110EibNycl...,156839,height
1,2,https://m.media-amazon.com/images/I/11TU2clswz...,792578,height
2,3,https://m.media-amazon.com/images/I/11gHj8dhhr...,792578,height
3,4,https://m.media-amazon.com/images/I/11lshEUmCr...,156839,height
4,5,https://m.media-amazon.com/images/I/21-LmSmehZ...,478357,height


In [4]:
df.isnull().sum()

,0
index,0
image_link,0
group_id,0
entity_name,0


Extracting all values in image_link column in to an array of image_links

In [5]:
df['image_link'].duplicated().sum()

0

In [6]:
image_links = df['image_link'].values

In [7]:
len(image_links)

500

In [8]:

def common_mistake(unit):
    if unit in constants.allowed_units:
        return unit
    if unit.replace('ter', 'tre') in constants.allowed_units:
        return unit.replace('ter', 'tre')
    if unit.replace('feet', 'foot') in constants.allowed_units:
        return unit.replace('feet', 'foot')
    return unit

def parse_string(s):
    s_stripped = "" if s==None or str(s)=='nan' else s.strip()
    if s_stripped == "":
        return None, None
    pattern = re.compile(r'^-?\d+(\.\d+)?\s+[a-zA-Z\s]+$')
    if not pattern.match(s_stripped):
        raise ValueError("Invalid format in {}".format(s))
    parts = s_stripped.split(maxsplit=1)
    number = float(parts[0])
    unit = common_mistake(parts[1])
    if unit not in constants.allowed_units:
        raise ValueError("Invalid unit [{}] found in {}. Allowed units: {}".format(
            unit, s, constants.allowed_units))
    return number, unit



In [9]:

def create_placeholder_image(image_save_path):
    try:
        placeholder_image = Image.new('RGB', (100, 100), color='black')
        placeholder_image.save(image_save_path)
    except Exception as e:
        return


In [10]:

def download_image(image_link, save_folder, retries=3, delay=3):
    if not isinstance(image_link, str):
        return

    filename = Path(image_link).name
    image_save_path = os.path.join(save_folder, filename)

    if os.path.exists(image_save_path):
        return

    for _ in range(retries):
        try:
            urllib.request.urlretrieve(image_link, image_save_path)
            return
        except:
            time.sleep(delay)

    create_placeholder_image(image_save_path) #Create a black placeholder image for invalid links/images


In [11]:

def download_images(image_links, download_folder, allow_multiprocessing=True):
    if not os.path.exists(download_folder):
        os.makedirs(download_folder)

    if allow_multiprocessing:
        download_image_partial = partial(
            download_image, save_folder=download_folder, retries=3, delay=3)

        with multiprocessing.Pool(64) as pool:
            list(tqdm(pool.imap(download_image_partial, image_links), total=len(image_links)))
            pool.close()
            pool.join()
    else:
        for image_link in tqdm(image_links, total=len(image_links)):
            download_image(image_link, save_folder=download_folder, retries=3, delay=3)


In [12]:
download_images(image_links, "/content/drive/MyDrive/Colab Notebooks/git/images")

100%|██████████| 500/500 [00:05<00:00, 93.71it/s] 
